In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
import CorrHelpers as ch

In [4]:
baseDir = os.getcwd()
dataLocation = 'data/Master_Data.csv'
dataFilePath = os.path.join(baseDir, dataLocation)

In [5]:
roughData = pd.read_csv(dataFilePath, index_col=0)

In [6]:
# Clean data by removing stocks that dont have history
# to 2005
masterData = roughData.dropna(axis=1)

In [7]:
byStockAndYear = ch.ByStockAndYear(masterData)

In [ ]:
byStockAndYear['MSFT'].keys()

In [8]:
# Drop 2019 data
for _, data in byStockAndYear.items():
    del data[2019]

# Create Rolling Correlation Data

In [ ]:
rollCorr_60 = ch.RollCorr(byStockAndYear, period=60)

In [9]:
import csv

In [ ]:
# Save data to file.
Saver(rollCorr_60)

In [10]:
def Saver(dictex):
    '''
    Code to save rolling correlation data to a csv file.
    from stack overflow: 
    https://stackoverflow.com/questions/50786266/writing-dictionary-of-dataframes-to-file
    '''
    for key, val in dictex.items():
        val.to_csv(f'PrivateData/RollCorr_60/RollCorr60_{key}.csv')
    with open('PrivateData/RollCorr_60/keys.txt', 'w') as f:
        f.write(str(list(dictex.keys())))
def Loader():
    with open('PrivateData/RollCorr_60/keys.txt', 'r') as f:
        keys = eval(f.read())
    dictex = {}
    for key in keys:
        dictex[key] = pd.read_csv(f'PrivateData/RollCorr_60/RollCorr60_{key}.csv', index_col=[0,1])
    return dictex

In [11]:
# Load Roll Corrolation data after restarting kernal.
rollCorr_60 = Loader()

# Create Seasonal Correlation Data

In [12]:
seasonalCorr_60 = ch.SeasonCorrTest(rollCorr_60, dropNum=10, n=0.75)

# Filter Seasonal Correlation Data for days with High Correlation

In [15]:
daysWithHighCorr_60 = ch.HighCorrDays(seasonalCorr_60)

# Calculate % Change for periods of high correlation

In [16]:
pctChangeResults_60 = ch.PctReturnForDays(daysWithHighCorr_60, byStockAndYear, periods=60)

# Executive Summary

In [17]:
execSummary_60 = ch.ExecSummaryCorr(pctChangeResults_60)

In [ ]:
# to save out put to file:
'''
for stock, days in execSummary_60.items():
    for day, data in days.items():
        dayValue = int(day[3:])
        if int(day[3:]) > 200:
            if (data['NumPos'] > 10) or (data['NumPos'] < 4):
                with open('PrivateData/ExecSummary_60.txt', 'a') as f:
                    f.write(f'{stock} {day} start:{dayValue-60} \n {data}\n')
                    f.write('\n')
'''            


# Let's look for seasonality at the turn of the year

In [42]:
loopYear = ch.LoopYear(byStockAndYear)

TypeError: unhashable type: 'list'

In [26]:
for k, v in loopYear.items():
    v.dropna(inplace=True)

In [ ]:
loopYear['MSFT'].loc[240:]

In [29]:
loopRollCorr_60 = ch.RollCorr(loopYear, period=60)

In [31]:
loopSeasonalCorr_60 = ch.SeasonCorrTest(loopRollCorr_60, dropNum=10, n=0.75)

In [32]:
loopDaysWithHighCorr = ch.HighCorrDays(loopSeasonalCorr_60)

In [35]:
loopDaysWithHighCorr['MSFT']

[244, 245]

In [77]:
def PctReturnForDays(data, pxData, periods):
    request = {}
    count=0
    if count > 5:
        return request
    for stock, data in data.items():
        requestValue = {}
        for k, day in enumerate(data):
            dataValue = {}
            # px at the day at which the high correlation occured
            end = pxData[stock].loc[day]
            # px N days prior to end day
            if (day-periods) == 0:
                start = pxData[stock].loc[(day-periods+1)]
                dataValue['StartDay'] = day-periods+1
            elif day < periods:
                remainder = periods-day
                # find max of index for pxData.
                endYearDay = max(pxData[stock].index)
                i = endYearDay - remainder
                start = pxData[stock].loc[i]
                dataValue['StartDay'] = i
            else:
                start = pxData[stock].loc[(day-periods)]
                dataValue['StartDay'] = day-periods
            pctChange = (end-start) / start
            dataValue['AvgReturn'] = round(pctChange.mean()*100, 2)
            dataValue['ReturnDetails'] = round(pctChange*100, 2)
            requestValue[f'Day{day}'] = dataValue
        request[stock] = requestValue
        count += 1
    return request

In [62]:
day = 46
period = 60
toZero = 60-46
finish = max(loopYear['NFLX'].index)
i = finish-toZero
print(i)

236


In [78]:
loopPctChangeOfResults = PctReturnForDays(loopDaysWithHighCorr, loopYear, periods=60)

In [ ]:
for day, details in loopPctChangeOfResults['NFLX'].items():
    print(details['StartDay'])

In [83]:
def ExecSummaryCorr(data, printupdate=False):
    request = {}
    status = 0
    outOf = len(data.keys())
    for stock, days in data.items():
        if printupdate:
            print(f'{status}/{outOf}', end=' | ')
            status += 1
        if len(days) >= 1:
            requestValue = {}
            for day, details in days.items():
                value = {}
                data = details['ReturnDetails']
                posTest = data > 0
                daysPos = data[posTest].count()
                daysNeg = data.count() - daysPos
                value['StartDay'] = details['StartDay']
                value['TotalTrades'] = data.count()
                value['NumPos'] = daysPos
                value['NumNeg'] = daysNeg
                value['AvgReturnOnPos'] = round(data[posTest].mean(), 2)
                value['AvgReturnOnNeg'] = round(data[data < 0].mean(), 2)
                requestValue[day] = value
            request[stock] = requestValue
    if printupdate:
        print()
    return request

In [84]:
loopExecSummary = ExecSummaryCorr(loopPctChangeOfResults)

In [94]:
# Save to txt file
for stock, days in loopExecSummary.items():
    for day, data in days.items():
        if (data['NumPos'] >= 10) or (data['NumPos'] <= 4):
            print(f'{stock} from: {data["StartDay"]} to: {day}')
            print(f'Total Trades:{data["TotalTrades"]}', end='')
            print(f'Number Positive: {data["NumPos"]}')
'''            
for stock, days in loopExecSummary.items():
    for day, data in days.items():
        dayValue = int(day[3:])
        if (int(day[3:]) > 245) or :
            if (data['NumPos'] > 10) or (data['NumPos'] < 4):
                with open('PrivateData/ExecSummary_60.txt', 'a') as f:
                    f.write(f'{stock} {day} start:{dayValue-60} \n {data}\n')
                    f.write('\n')
'''

MSFT from: 184 to: Day244
Total Trades:13Number Positive: 11
MSFT from: 185 to: Day245
Total Trades:13Number Positive: 11
GOOG from: 154 to: Day214
Total Trades:13Number Positive: 10
GOOG from: 156 to: Day216
Total Trades:13Number Positive: 10
GOOG from: 157 to: Day217
Total Trades:13Number Positive: 10
GOOG from: 163 to: Day223
Total Trades:13Number Positive: 10
GOOG from: 168 to: Day228
Total Trades:13Number Positive: 10
GOOG from: 169 to: Day229
Total Trades:13Number Positive: 10
GOOG from: 170 to: Day230
Total Trades:13Number Positive: 10
GOOG from: 171 to: Day231
Total Trades:13Number Positive: 10
GOOG from: 174 to: Day234
Total Trades:13Number Positive: 10
GOOG from: 178 to: Day238
Total Trades:13Number Positive: 10
GOOG from: 179 to: Day239
Total Trades:13Number Positive: 10
GOOGL from: 154 to: Day214
Total Trades:13Number Positive: 10
GOOGL from: 156 to: Day216
Total Trades:13Number Positive: 10
GOOGL from: 157 to: Day217
Total Trades:13Number Positive: 10
GOOGL from: 168 to: D

"            \nfor stock, days in loopExecSummary.items():\n    for day, data in days.items():\n        dayValue = int(day[3:])\n        if (int(day[3:]) > 245) or :\n            if (data['NumPos'] > 10) or (data['NumPos'] < 4):\n                with open('PrivateData/ExecSummary_60.txt', 'a') as f:\n                    f.write(f'{stock} {day} start:{dayValue-60} \n {data}\n')\n                    f.write('\n')\n"

In [ ]:
# prints days that are within the requested range

for stock, days in loopExecSummary.items():
    for day, data in days.items():
        dayValue = int(day[3:])
        if (dayValue > 230) or (dayValue < 100):
            print(stock, day)
            print(data)